In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt


In [27]:

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

train_images, test_images = train_images / 255.0, test_images / 255.0
train_images_expanded = tf.expand_dims(train_images, -1)
test_images_expanded = tf.expand_dims(test_images, -1)

In [28]:

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

In [29]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

In [30]:

for layer in base_model.layers[:15]:
    layer.trainable = False

In [32]:

def preprocess_and_augment(images, labels, batch_size=64):
    def _process_image(image, label):
        image = tf.image.resize(image, (128, 128))
        image = tf.image.grayscale_to_rgb(image)
        return image, label

    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(_process_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset


train_dataset_vgg = preprocess_and_augment(train_images_expanded, train_labels)
test_dataset_vgg = preprocess_and_augment(test_images_expanded, test_labels)


def build_vgg16_model():
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),  
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

vgg16_model = build_vgg16_model()

history_vgg16 = vgg16_model.fit(
    train_dataset_vgg,
    epochs=1,  
    validation_data=test_dataset_vgg
)

test_loss_vgg16, test_acc_vgg16 = vgg16_model.evaluate(test_dataset_vgg, verbose=2)
print(f'\nTest accuracy on VGG16 model: {test_acc_vgg16}')

938/938 ━━━━━━━━━━━━━━━━━━━━ 3627s 4s/step - accuracy: 0.8321 - loss: 0.4964 - val_accuracy: 0.8865 - val_loss: 0.3203
157/157 - 561s - 4s/step - accuracy: 0.8865 - loss: 0.3203

Test accuracy on VGG16 model: 0.8865000009536743


In [34]:
vgg16_model.save('vgg16_model.keras')